In [ ]:
import h5py
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import dask.array as da
import os
import sys
import logging


logging.getLogger('matplotlib.font_manager').disabled = True
numba_logger = logging.getLogger('numba')
numba_logger.setLevel(logging.WARNING)

matplotlib_logger = logging.getLogger('matplotlib')
matplotlib_logger.setLevel(logging.WARNING)

src_path = os.path.abspath(os.path.join(os.getcwd(), '..', '..'))
print(src_path)
sys.path.append(src_path)

from src.Analysis import Analysis, AnalysisManager, GR_Confirmation, SpotDetection_Confirmation

Loads in the data from specified location

In [2]:
# loc = r"Y:/smFISH_images/Eric_smFISH_images/20230511/GR_ICC_3hr_R1_1nM_10min_050823/GR_ICC_3hr_R1_1nM_10min_050823.h5"
am = AnalysisManager(log_location=r'Y:\Users\Jack\All_Analysis')

In [ ]:
# list all analysis done 
am.list_analysis_names()

In [ ]:
# can filter on name and dates
am.select_analysis('ER_Dec0324_GR')
print(am.analysis)

Does analysis/confirmation

In [5]:
# select GR conformation analysis 
GR = GR_Confirmation(am)

In [6]:
# this loads the data into memory 
GR.get_data()

In [7]:
# This is a temporary fix, I didnt do this in the correct spot in the first run
GR.cellprops['fov'] = pd.Categorical(GR.cellprops['fov']).codes # this will be fixed in future runs


In [ ]:
# make sure this has all the measurements you want
GR.cellprops.keys()

In [ ]:
# run this multiple times to see a new randomly selected cell
GR.display()